In [ ]:
from cml.workers_v1 import launch_workers
import subprocess, socket, os, sys
from subprocess import call 

NUM_WORKERS = 3
worker_cpu = 4
worker_memory = 32
worker_gpu = 1
hostfile = "/home/cdsw/hostfile.txt"

def display_file_content(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        print(content)

#preparing hostfile with pdsh/openMPI specific format
def redirect_to_file(text):
    file = open(hostfile, 'a')
    file.write(text + " slots=1\n")
    file.close
    
for i in range(NUM_WORKERS):
    worker_cmd = "python worker_p.py"
    print(f"Launch CML worker pod {i}...")
    # worker0 runs a different script
    if i == 0:
        with open('/home/cdsw/hostfile.txt', 'w') as f_obj:
            call(['python', 'master_p.py'], stdout=f_obj)
    else:
        launch_workers(name=f'CML Worker Pods {i}', n=1, cpu=worker_cpu, memory=worker_memory, nvidia_gpu = worker_gpu,  code="!"+worker_cmd )
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.bind(("0.0.0.0", 6000))
        s.listen(1)    
        conn, addr = s.accept()
        for i in range(2):
            data = conn.recv(20)
            if not data: break
            string = str(data, encoding='utf-8')
            redirect_to_file(string)
            conn.send("Hello From Server!".encode())
        conn.close()

print("Content of hostfile:")
display_file_content(hostfile)

In [ ]:
!export PDSH_SSH_ARGS_APPEND='';deepspeed --hostfile /home/cdsw/hostfile.txt \
--launcher pdsh \
--num_nodes 3 \
--num_gpus 1 \
--master_addr $CDSW_IP_ADDRESS \
--ssh_port 2222 textsql_train.py \
--model_id 't5-small' \
--outputdir ds-zero1-t5small \
--epochs 3 \
--per_device_train_batch_size 32 \
--per_device_eval_batch_size 32 \
--deepspeed dsconfig/zero1profiler.json

In [ ]:
!export PDSH_SSH_ARGS_APPEND='';deepspeed --hostfile /home/cdsw/hostfile.txt \
--launcher pdsh \
--num_nodes 3 \
--num_gpus 1 \
--master_addr $CDSW_IP_ADDRESS \
--ssh_port 2222 textsql_train.py \
--model_id 't5-large' \
--outputdir ds-trainoutput-wikisql \
--epochs 3 \
--gradient_checkpointing False \
--per_device_train_batch_size 32 \
--per_device_eval_batch_size 32 \
--deepspeed dsconfig/zero1profiler.json

In [ ]:
!export PDSH_SSH_ARGS_APPEND='';deepspeed --hostfile /home/cdsw/hostfile.txt \
--launcher pdsh \
--num_nodes 2 \
--num_gpus 1 \
--master_addr $CDSW_IP_ADDRESS \
--ssh_port 2222 textsql_train.py \
--model_id 't5-large' \
--outputdir ds-trainoutput-wikisql \
--epochs 3 \
--gradient_checkpointing True \
--per_device_train_batch_size 32 \
--per_device_eval_batch_size 32 \
--deepspeed dsconfig/zero1profiler.json

In [ ]:
!export PDSH_SSH_ARGS_APPEND='';deepspeed --hostfile /home/cdsw/hostfile.txt \
--launcher pdsh \
--num_nodes 3 \
--num_gpus 1 \
--master_addr $CDSW_IP_ADDRESS \
--ssh_port 2222 textsql_train.py \
--model_id 't5-3b' \
--outputdir ds-train-zero3-large \
--epochs 3 \
--gradient_checkpointing False \
--per_device_train_batch_size 1 \
--per_device_eval_batch_size 1 \
--deepspeed dsconfig/zero3profiler.json